# Pima Indians Diabetes Dataset: 
## `Diabetes.csv`
---
## Context
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective is to predict based on diagnostic measurements whether a patient has diabetes.

## Content
Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

|Variable | Description |
|---|---|
|`Pregnancies` | Number of times pregnant |
|`Glucose`| Plasma glucose concentration a 2 hours in an oral glucose tolerance test |
|`BloodPressure` | Diastolic blood pressure (mm Hg) |
|`SkinThickness` | Triceps skin fold thickness (mm) |
|`Insulin` | 2-Hour serum insulin (mu U/ml) |
|`BMI` | Body mass index (weight in kg/(height in m)^2) |
|`DiabetesPedigreeFunction` | Diabetes pedigree function |
|`Age` | Age (years) |
|`Outcome` | Class variable (0 or 1) |

---

*Sources:*
* *Original owners: National Institute of Diabetes and Digestive and Kidney Diseases*
* *Donor of database: Vincent Sigillito (vgs@aplcen.apl.jhu.edu) Research Center, RMI Group Leader,  Applied Physics Laboratory, The Johns Hopkins University*

*Original Date Received: 9 May 1990* 

*Retrieved from Kaggle 22 AUG 2021*

In [25]:
import pandas as pd
import numpy as np
from glob import glob
import pandas_profiling

diabetes = pd.read_csv('data/diabetes.csv')

## EDA

---

In [2]:
if len(glob('profiling/diabetes.html')) == 0:
    diabetes.profile_report().to_file('profiling/diabetes.html')

---

Issues Observed:
* No missing values or duplicate rows. **Awesome!**
* 8 numeric variables and 1 categorical.
* Some zeroes in `BloodPressure`, `BMI` - not compatible with life. Impute values.
* A lot of zeroes in `SkinThickness` - assumed not measured. Impute values.
* A lot of zeroes in `Insulin` - assumed not taking insulin. Leave for now.
* Potential correlation between `Age` and `Glucose`. This may be due to zeros, but inclined to leave alone.
* As above for `Insulin` and `SkinThickness`.


This step replaces the zeroes with imputed values. Tried means first, but medians is better for skew distro.

In [3]:
diabetes['BloodPressure'] = diabetes['BloodPressure'].replace(0, diabetes[diabetes['BloodPressure'] > 0].BloodPressure.median())
diabetes['BMI'] = diabetes['BMI'].replace(0, diabetes[diabetes['BMI'] > 0].BloodPressure.mean())
diabetes['SkinThickness'] = diabetes['SkinThickness'].replace(0, diabetes[diabetes['SkinThickness'] > 0].BloodPressure.median())

Quick check imputation has worked.

---

In [4]:
if len(glob('profiling/diabetes_post_impute.html')) == 0:
    diabetes.profile_report().to_file('profiling/diabetes_post_impute.html')

---

All good.

## Train-Test Splitting 
Let's stratify by diabetes outcome so that there's equal proportions for train and test.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(['Outcome'], axis = 1), 
                                                    diabetes['Outcome'], 
                                                    test_size = 0.4, 
                                                    random_state = 42, 
                                                    stratify = diabetes['Outcome'])

---

## Fitting the Logistic Regression Model

This is a supervised binary classification problem. Logistic regression is a pretty good first model to try (and familiar to a statistician).

*Why LogisticRegressionCV*

Here 100 iterations isn't enough, so let's set `max_iter = 1000`. The other parameters `n_jobs = -1` uses all the CPU cores available to train (but there's only two on this VM and it doesn't take long to train anyway).

In [6]:
from sklearn.linear_model import LogisticRegressionCV

LogReg = LogisticRegressionCV(cv = 10, max_iter = 1000, n_jobs = -1)
LogReg.fit(X_train, y_train)

LogisticRegressionCV(cv=10, max_iter=1000, n_jobs=-1)

In [7]:
LogReg.score(X_train, y_train) # Mean Accuracy = (TP + TN) / (P + N)

0.7804347826086957

In [8]:
LogReg.score(X_test, y_test)

0.737012987012987

Both train and test \>70% accurate out of the box. That's not bad.

## Adding StandardScaler
Scales via: $ Z = \frac {(x_i - \mu)} {\sigma}$ this should stop factors with larger scales or variance unduly influencing fitting.

A small improvement.

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pl_s = Pipeline([("Scaler", StandardScaler()), 
               ("LogReg", LogisticRegressionCV(cv = 10, max_iter = 5000, n_jobs = -1))
              ])
              

pl_s.fit(X_train, y_train)

display("Train: " + str(pl_s.score(X_train, y_train))[0:4])
display("Test: " + str(pl_s.score(X_test, y_test))[0:4])

'Train: 0.78'

'Test: 0.75'

## Adding Interactions

Overfitting in practice. Up to $3^{rd}$ order interactions ...

In [10]:
from sklearn.preprocessing import PolynomialFeatures

pl_s_3 = Pipeline([("Scaler", StandardScaler()), 
                 ("Poly", PolynomialFeatures(degree = 3, interaction_only = True)), 
                 ("LogReg", LogisticRegressionCV(cv = 10, max_iter = 5000, n_jobs = -1))
              ])
              

pl_s_3.fit(X_train, y_train)

display("Train: " + str(pl_s_3.score(X_train, y_train))[0:4])
display("Test: " + str(pl_s_3.score(X_test, y_test))[0:4])

'Train: 0.85'

'Test: 0.69'

Drop to $2^{nd}$ order interactions ...

In [11]:
pl_s_2 = Pipeline([("Scaler", StandardScaler()), 
                 ("Poly", PolynomialFeatures(degree = 2, interaction_only = True)), 
                 ("LogReg", LogisticRegressionCV(cv = 10, max_iter = 5000, n_jobs = -1))
              ])
              

pl_s_2.fit(X_train, y_train)

display("Train: " + str(pl_s_2.score(X_train, y_train))[0:4])
display("Test: " + str(pl_s_2.score(X_test, y_test))[0:4])

'Train: 0.80'

'Test: 0.72'

Try *l1* regularisation...

In [12]:
pl_s_2_l1 = Pipeline([("Scaler", StandardScaler()), 
                 ("Poly", PolynomialFeatures(degree = 2, interaction_only = True)), 
                 ("LogReg", LogisticRegressionCV(cv = 10, 
                                                 max_iter = 
                                                 5000, n_jobs = -1, 
                                                 penalty = 'l1', 
                                                 solver = 'saga'))
              ])
              

pl_s_2_l1.fit(X_train, y_train)

display("Train: " + str(pl_s_2_l1.score(X_train, y_train))[0:4])
display("Test: " + str(pl_s_2_l1.score(X_test, y_test))[0:4])

'Train: 0.77'

'Test: 0.74'

---
## Support Vector Classification

In [61]:
from sklearn.svm import SVC

from sklearn.preprocessing import PolynomialFeatures

pl_svc = Pipeline([("Scaler", StandardScaler()),
                 ("SVC", SVC())
              ])
              

pl_svc.fit(X_train, y_train)

display("Train: " + str(pl_svc.score(X_train, y_train))[0:4])
display("Test: " + str(pl_svc.score(X_test, y_test))[0:4])

'Train: 0.84'

'Test: 0.73'

Testing not great out of the box ... maybe some value in tuning regularization strength and try a few kernels.

In [67]:
from sklearn.model_selection import GridSearchCV

search_params = {'SVC__C': np.linspace(0.1,2,20), 
                 'SVC__kernel': ['linear','poly', 'rbf']}

search = GridSearchCV(pl_svc, search_params)
search.fit(X_train, y_train)

display(search.best_params_)
display("Train: " + str(search.best_score_)[0:4])

{'SVC__C': 0.7, 'SVC__kernel': 'rbf'}

'Train: 0.79'

In [65]:
# Test train at C = 0.7

pl_svc = Pipeline([("Scaler", StandardScaler()),
                 ("SVC", SVC(C=0.7))
              ])
              

pl_svc.fit(X_train, y_train)
display("Test: " + str(pl_svc.score(X_test, y_test))[0:4])

# Not much better than log-reg.

'Test: 0.74'

Now we have a feel for using sci-kit learn using the data, let's build a framework to train, tune and evaluate a few sensible models and compare. Over to the `pima_diabetes_framework.ipynb` for more ...